# Convert NeuCosmA files
The file format is as following (by column):

1. parent id
2. daughter id
3. systematic flag (currently ignored)
4. log10(E [GeV]) (E or eps_r or y depending on corresp. column)
5. g_ij(y) [mubarn = 10^-30 cm^2]
6. M_ij(eps_r)
7. f_i(y) [mubarn = 10^-30 cm^2]
8. sigma_i(eps_r) [mubarn = 10^-30 cm^2]

In [1]:
import numpy as np
import pandas as pd

In [2]:
data_table = pd.read_csv('./prince_data_utils/resources/photo-nuclear/160513_XDIS_PSB-SS_syst.dat',delim_whitespace=True,
            names=['mother','daughter','sys_flag','log10(E)','g(y)','M(eps)','f_i(y)','sigma_i'])
data_table = data_table.set_index(['mother','daughter'])
data_table['dsigma_ij'] = data_table['sigma_i'] * data_table['M(eps)'] 
data_table

sys_flag      log10(E)     g(y)  M(eps)    f_i(y)  sigma_i  \
mother daughter                                                               
201    100              0 -3.500000e+00   0.0000     0.0    0.0000      0.0   
       100              0 -3.450000e+00   0.0000     0.0    0.0000      0.0   
       100              0 -3.400000e+00   0.0000     0.0    0.0000      0.0   
       100              0 -3.350000e+00   0.0000     0.0    0.0000      0.0   
       100              0 -3.300000e+00   0.0000     0.0    0.0000      0.0   
...                   ...           ...      ...     ...       ...      ...   
5626   5525             1 -2.000000e-01  18.0244     0.0  140.5530      0.0   
       5525             1 -1.500000e-01  16.0738     0.0  100.1920      0.0   
       5525             1 -1.000000e-01  15.9104     0.0   92.9049      0.0   
       5525             1 -5.000000e-02  12.2481     0.0   46.0163      0.0   
       5525             1  1.928650e-16  12.0852     0.0   34.6745      0.0   

                 dsigma_ij  
mother daughter             
201    100             0.0  
       100             0.0  
       100             0.0  
       100             0.0  
       100             0.0  
...                    ...  
5626   5525            0.0  
       5525            0.0  
       5525            0.0  
       5525            0.0  
       5525            0.0  

[48493 rows x 7 columns]

# Old function

In [3]:
def _load_NEUCOSMA_file(filename):
        """Loads a txt file with format as define in the internal note.

        Args:
            filename (string): name of the ile including path

        Returns:
            (filename1, filename2) Two pickled dictionaries saved on the
            same directory as `filename` which contain:
            filename1: a dictionary indexed (mother, daughter) where the
            the g function and the multiplicity are stored.
            filename2: a dictionary indexed (mother) where the f function
            and the total inelasticcross section are stored.
        """

        # The file format is as following (by column):
        # 1. parent id
        # 2. daughter id
        # 3. systematic flag (currently ignored)
        # 4. log10(E [GeV]) (E or eps_r or y depending on corresp. column)
        # 5. g_ij(y) [mubarn = 10^-30 cm^2]
        # 6. M_ij(eps_r)
        # 7. f_i(y) [mubarn = 10^-30 cm^2]
        # 8. sigma_i(eps_r) [mubarn = 10^-30 cm^2]

        with open(filename) as f:
            text_data = f.readlines()

        # We need the following: sigma_i, sigma_ij = M_ij * sigma_i

        mo, da = (int(l) for l in text_data[0].split()[:2])
        cs_nonel, cs_incl = {}, {}
        e, g, mu, f, cs = (), (), (), (), ()

        neucosma_data = {}
        neucosma_data['f'] = {}
        neucosma_data['g'] = {}
        neucosma_data['m'] = {}

        for line in text_data:
            m, d, _, e_k, g_ijk, m_ijk, f_ik, cs_ik = line.split()

            m, d = int(m), int(d)
            if d == da:
                e += (float(e_k), )
                g += (float(g_ijk), )
                mu += (float(m_ijk), )
                f += (float(f_ik), )
                cs += (float(cs_ik), )
            else:
                neucosma_data['g'][mo, da] = np.array(g)  # stored in mubarn
                neucosma_data['m'][mo, da] = np.array(mu)
                # Factor 1e30 below, for conversion to cm-2
                cs_incl[mo, da] = np.array(mu) * np.array(cs) * 1e-30
                # reset values of lists
                da = d
                if m != mo:
                    neucosma_data['f'][mo] = np.array(f)
                    cs_nonel[mo] = np.array(cs) * 1e-30  # conversion to cm-2
                    mo = m
                e, g, mu, f, cs = (float(e_k), ), (float(g_ijk), ), \
                                  (float(m_ijk), ), (float(f_ik), ),\
                                  (float(cs_ik), )

        # Do not forget the last mother:
        neucosma_data['f'][mo] = np.array(f)
        cs_nonel[mo] = np.array(cs) * 1e-30  # conversion to cm-2

        neucosma_data['g'][mo, da] = np.array(g)  # stored in mubarn
        neucosma_data['m'][mo, da] = np.array(mu)
        # Factor 1e30 below, for conversion to cm-2
        cs_incl[mo, da] = np.array(mu) * np.array(cs) * 1e-30

        # If proton and neutron cross sections are not in contained
        # in the files, set them to 0. Needed for TALYS and CRPropa2 and PSB
        for pid in [101, 100]:
            if pid not in cs_nonel:
                cs_nonel[pid] = np.zeros_like(e)

        print('known species after loading NeuCosmA file:')
        print(np.sort(list(cs_nonel.keys())))

        return 10**np.array(e), cs_nonel, cs_incl

In [4]:
egrid_old, nonel_old, incl_old = _load_NEUCOSMA_file('./prince_data_utils/resources/photo-nuclear/160513_XDIS_PSB-SS_syst.dat')

known species after loading NeuCosmA file:
[ 100  101  201  302  402  904 1005 1105 1206 1306 1407 1507 1608 1708
 1808 1909 2010 2110 2210 2311 2412 2512 2612 2713 2814 2914 3014 3115
 3216 3316 3416 3517 3618 3718 3818 3919 4019 4119 4220 4320 4420 4521
 4622 4722 4822 4922 5023 5123 5224 5324 5424 5525 5626]


# Consistency checks

Before converting, we need to check two things:

- The energy grid is the same for all channels (not guaranteed for NEUCOSMA, but requried for PriNCe)
- The nonel cross section should be the same for all mother (It is stored redundantly)

In [5]:
last_idx = None
last_egrid = None
last_csec  = None
for index, data in data_table.groupby(level=[0,1]):
    if last_idx is None:
        last_idx = index
        last_egrid = data['log10(E)']
        last_csec = data['sigma_i']

    assert np.all(last_egrid.values == data['log10(E)'].values)
    if index[0] == last_idx[0]:
        assert np.all(last_csec.values == data['sigma_i'].values)
 
    last_idx = index
    last_egrid = data['log10(E)']
    last_csec = data['sigma_i']   
        
    #print(data['sigma_i']) 

# Now write this to datafiles

In [6]:
egrid_peanut = np.loadtxt('./prince_data_utils/resources/photo-nuclear/PEANUT_IAS_egrid.dat.bz2')
nonel_peanut = np.loadtxt('./prince_data_utils/resources/photo-nuclear/PEANUT_IAS_nonel.dat.bz2')
incl_peanut  = np.loadtxt('./prince_data_utils/resources/photo-nuclear/PEANUT_IAS_incl_i_j.dat.bz2')

print(egrid_peanut.shape)
print(nonel_peanut.shape)
print(incl_peanut.shape)

(148,)
(405, 149)
(49386, 150)


In [7]:
egrid = 10**data_table[['log10(E)']].drop_duplicates()
egrid = egrid.values * 1e3 # energy conversion to MeV
egrid = egrid.flatten()

print(egrid.shape)
np.savetxt('./prince_data_utils/resources/photo-nuclear/PSB_egrid.dat.bz2',egrid)

(71,)


In [8]:
table = data_table.reset_index()
table = table[['mother','log10(E)','sigma_i']].drop_duplicates()
table = table.drop('log10(E)',axis='columns')

table = table.set_index('mother')
print(table.shape)

array = []
for index, data in table.groupby(level=[0]):
    data = data.T.values.flatten() * 1e-3 # mubarn to mbarn
    array.append(np.concatenate([np.array(index)[None],data]))   
nonel = np.array(array)
print(nonel.shape)
np.savetxt('./prince_data_utils/resources/photo-nuclear/PSB_nonel.dat.bz2',nonel)

(3621, 1)
(51, 72)


In [9]:
table = data_table.reset_index()
table = table[['mother','daughter','log10(E)','dsigma_ij']].drop_duplicates()
table = table.drop('log10(E)',axis='columns')

table = table.set_index(['mother','daughter'])
print(table.shape)

array = []
for index, data in table.groupby(level=[0,1]):
    data = data.T.values.flatten() * 1e-3 # mubarn to mbarn
    array.append(np.concatenate([np.array(index),data]))
incl = np.array(array)
print(incl.shape)
np.savetxt('./prince_data_utils/resources/photo-nuclear/PSB_incl_i_j.dat.bz2',incl)

(48493, 1)
(683, 73)


# Testing

In [10]:
np.allclose(egrid*1e-3,egrid_old,atol=0.,rtol=1e-15) # The units are different (i.e. MeV vs GeV)

True

In [11]:
for line in nonel:
    mo = int(line[0])
    data = line[1:]
    #print(mo)
    assert np.allclose(data*1e-27,nonel_old[mo],atol=0.,rtol=1e-15)

In [12]:
for line in incl:
    mo, da = int(line[0]),int(line[1])
    data = line[2:]
    # print(mo,da)
    assert np.allclose(data*1e-27,incl_old[mo,da],atol=0.,rtol=1e-15)
    # print((data*1e-27-incl_old[mo,da])/incl_old[mo,da])

In [13]:
len(nonel)

51

In [14]:
len(nonel_old)

53

In [15]:
len(incl)

683

In [16]:
len(incl_old)

683